# Projeto Ciencia de Dados: Carteira de Ações com Inteligência Artificial

- Projeto Inspiração: https://medium.com/swlh/teaching-a-machine-to-trade-stocks-like-warren-buffett-part-i-445849b208c6
- Resumo Fundamentos Empresas BR: https://fundamentus.com.br/index.php

### Criar Dicionários de Dataframes com cada empresa

In [ ]:
import pandas as pd
import os

# Lista de empresas
empresas = ["ABEV3", "AZUL4", "BTOW3", "B3SA3", "BBSE3", "BRML3", "BBDC4", "BRAP4", "BBAS3", "BRKM5", 
            "BRFS3", "BPAC11", "CRFB3", "CCRO3", "CMIG4", "HGTX3", "CIEL3", "COGN3", "CPLE6", "CSAN3", 
            "CPFE3", "CVCB3", "CYRE3", "ECOR3", "ELET6", "EMBR3", "ENBR3", "ENGI11", "ENEV3", "EGIE3", 
            "EQTL3", "EZTC3", "FLRY3", "GGBR4", "GOAU4", "GOLL4", "NTCO3", "HAPV3", "HYPE3", "IGTA3", 
            "GNDI3", "ITSA4", "ITUB4", "JBSS3", "JHSF3", "KLBN11", "RENT3", "LCAM3", "LAME4", "LREN3", 
            "MGLU3", "MRFG3", "BEEF3", "MRVE3", "MULT3", "PCAR3", "PETR4", "BRDT3", "PRIO3", "QUAL3", 
            "RADL3", "RAIL3", "SBSP3", "SANB11", "CSNA3", "SULA11", "SUZB3", "TAEE11", "VIVT3", "TIMS3", 
            "TOTS3", "UGPA3", "USIM5", "VALE3", "VVAR3", "WEGE3", "YDUQ3"]

# Dicionário para armazenar os dados
fundamentos = {}

# Diretório dos arquivos
diretorio = "IA_carteira45Z/balancos-20240720T050244Z-001/balancos"

# Listar os arquivos no diretório
arquivos = os.listdir(diretorio)

# Loop através dos arquivos
for arquivo in arquivos:
    # Extrair o nome do arquivo
    nome = arquivo[-9:-4]
    if "11" in nome:
        nome = arquivo[-10:-4]
    
    # Verificar se o nome está na lista de empresas
    if nome in empresas:
        print(nome)
        # Construir o caminho completo para o arquivo
        caminho_arquivo = os.path.join(diretorio, arquivo)

        # Ler os dados dos arquivos Excel
        balanco = pd.read_excel(caminho_arquivo, sheet_name=0, engine='xlrd')
        dre = pd.read_excel(caminho_arquivo, sheet_name=1, engine='xlrd')

        # Manipulação dos DataFrames
        balanco.iloc[0, 0] = nome
        balanco.columns = balanco.iloc[0]
        balanco = balanco[1:]
        balanco = balanco.set_index(nome)
        
        dre.iloc[0, 0] = nome
        dre.columns = dre.iloc[0]
        dre = dre[1:]
        dre = dre.set_index(nome)

        # Adicionar ao dicionário fundamentos
        fundamentos[nome] = pd.concat([balanco, dre])

# Exibir o resultado
for empresa, dados in fundamentos.items():
    print(f"Empresa: {empresa}")
    display(dados)


### Pegar Preços das Ações nas Datas Correspondentes

In [ ]:
cotacoes_df = pd.read_excel("IA_carteira45Z/Cotacoes.xlsx")
cotacoes = {}
for empresa in cotacoes_df["Empresa"].unique():
    cotacoes[empresa] = cotacoes_df.loc[cotacoes_df['Empresa']==empresa, :]

print(len(cotacoes))

### Remover empresas que tem cotações vazias da análise (mesmo após o tratamento que fizemos na hora de pegar as cotações)

In [ ]:
for empresa in empresas:
    if cotacoes[empresa].isnull().values.any():
        cotacoes.pop(empresa)
        fundamentos.pop(empresa)
empresas = list(cotacoes.keys())
print(len(empresas))

### Juntar fundamentos com Preço da Ação

In [ ]:
# no cotacoes: jogar as datas para índice
# no fundamnetos:
    # trocar linhas por colunas
    # tratar as datas para formato de data do python
    # juntar os fundamentos com a coluna Adj Close das cotacoes
for empresa in fundamentos:
    tabela = fundamentos[empresa].T
    tabela.index = pd.to_datetime(tabela.index, format="%d/%m/%Y")
    tabela_cotacao = cotacoes[empresa].set_index("Date")
    tabela_cotacao = tabela_cotacao[["Adj Close"]]
    
    tabela = tabela.merge(tabela_cotacao, right_index=True, left_index=True)
    tabela.index.name = empresa
    fundamentos[empresa] = tabela
display(fundamentos["ABEV3"])

### Tratar colunas
    
1. Vamos pegar apenas empresas que possuem as mesmas colunas
2. Ajeitar colunas com nome repetido
3. Analisar valores vazios nas colunas

#### 1. Remover da análise colunas que não existem em alguma tabela

####  2. Ajeitando colunas com o mesmo nome

#### 3. Analisar valores vazios nas colunas

### Criando os rótulos: Comprar, Não Comprar ou Vender?

Não queremos saber quando vender, mas inclui essa categoria para conseguir identificar quando que o nosso modelo vai sugerir uma compra quando na verdade o melhor momento era vender. Isso significa que o modelo errou "mais" do que quando sugeriu comprar e simplesmente o certo era não comprar

Regra:
1. Subiu mais do que o Ibovespa (ou caiu menos) -> Comprar (Valor = 2)
2. Subiu menos do que o Ibovespa até Ibovespa - 2% (ou caiu mais do que Ibovespa até Ibovespa -2%) -> Não Comprar (Valor = 1)
3. Subiu menos do que o Ibovespa - 2% (ou caiu mais do que Ibovespa -2%) -> Vender (Valor = 0)

### Hora de tornar tudo 1 dataframe só

### Análise Exploratória

#### 1. Quantidade de Respostas em cada Tipo de Decisão

#### 2. Correlação

#### Vamos remover Todas as Colunas "já explicadas" pelo Ativo Total

### Vamos partir para Feature Selection

Será que todas essas features são importantes mesmo para o nosso modelo? Muitas features nem sempre é bom, se pudermos reduzir sem perder eficiência do nosso modelo, melhor

Aqui temos 2 alternativas:

1. Seguir com todas as features e depois tentar melhorar o nosso modelo
2. Usar algum critério para selecionar as melhores features para prever e criar o modelo a partir apenas dessa seleção menor de features

Vou seguir com a opção 2, porque é mais rápida e, caso dê certo, facilita a nossa vida. Se der errado, a gente volta aqui e refaz o processo

### Aplicação do StandardScaler para melhorar nossos modelos de MachineLearning

### Separação dos dados em treino e teste

### Criação de um Dummy Classifier (Uma baseline para ver se os nossos modelos são melhores do que puro chute)

### Métricas de Avaliação

- Precisão vai ser nossa métrica principal
- Recall pode ser útil, mas precisão no caso de ações é mt mais importante.

Explicação: Foto dos Gatos e Cachorros na Wikipedia: https://en.wikipedia.org/wiki/Precision_and_recall

### Modelos que vamos testar
- AdaBoost
- Decision Tree
- Random Forest
- ExtraTree
- Gradient Boost
- K Nearest Neighbors (KNN)
- Logistic Regression
- Naive Bayes
- Support Vector Machine (SVM)
- Rede Neural

### Agora vamos ao tunning do modelo

- é bom sempre incluir no tuning os parâmetros "padrões" do modelo, se não poder ser que vc só encontre resultados piores

### Vamos seguir então com o Modelo BLA tunado para ser o nosso modelo

- Agora ao que interessa...será que vamos conseguir escolher boas ações e ter uma carteira rentável?
- Para isso, temos que pegar o último Trimestre (que não usamos para treinar o modelo nem testar) e ver como ele teria saído até o momento em que gravo esse vídeo...

### Repetindo os passos com a base fundamentos só que pegando apenas o último tri de cada empresa

### Agora fazemos a previsão de decisões